# Draw with replacement

Imagin we have an urn, filled with numbered balls. We draw a number of these balls, but after each draw, we place the drawn ball back in the urn. How many different numbers will we have seen after a given number of draws?

To make this precise, imagine the urn contains $N$ balls, labeled with the numbers $0, 1, ..., N$. Let us refer to $N$ as the _urn size_. We draw $S$ balls, our _sample size_, but after every draw, we place the ball back in the urn. I.e. for each of the $S$ draws, we are effectively sampling from all of the $N$ balls. How many different numbers do we expect to have seen in a sample of size $S$?

In [1]:
import numpy as np

## Setting urn and sample size

In [2]:
N = 5  # Urn size
S = 5  # Sample size

## A first implementation
Before trying to calculate how many different numbers we expect to see, let us write a function which performs the experiment a given number of times, and computes the average.

In [14]:
def compute_expected_sample_size(urn_size: int, sample_size: int, no_samples: int = 1_000) -> float:
    """Computes the expected sample size, given the urn size and sample size, by simulating the 
    experiment ``no_samples`` times."""
    # The urn is represented as a list of numbers 0, 1, ..., urn_size - 1
    urn = list(range(urn_size))
    
    # Run the experiment
    diff_numbers_in_sample = []
    for _ in range(int(no_samples)):
        sample = np.random.choice(urn, sample_size, replace=True)
        diff_numbers = len(set(sample))
        diff_numbers_in_sample.append(diff_numbers)
        
    # Compute and return mean
    return np.mean(diff_numbers_in_sample)    

In [15]:
# Let us run a single experiment
%time print(compute_expected_sample_size(N, S))

3.366
CPU times: user 19.9 ms, sys: 7.99 ms, total: 27.9 ms
Wall time: 23.7 ms


## Making it a bit faster

In [44]:
def compute_expected_sample_size_fast(urn_size: int, sample_size: int, no_samples: int = 1_000) -> float:
    """Computes the expected sample size, given the urn size and sample size, by simulating the 
    experiment ``no_samples`` times."""
    # The urn is represented as a list of numbers 0, 1, ..., urn_size - 1
    urn = list(range(urn_size))
    
    # Run the experiment
    all_samples = np.random.choice(urn, (no_samples, sample_size), replace=True)
    
    # Compute length, return average
    def no_unique(arr: list) -> int:
        return len(set(arr))
    
    return np.mean(list(map(no_unique, all_samples)))

In [62]:
no_samples = 100000
N = 5
S = 5
%time print(f"{compute_expected_sample_size(N, S, no_samples):.10f}")
%time print(f"{compute_expected_sample_size_fast(N, S, no_samples):.10f}")

3.3629300000
CPU times: user 1.97 s, sys: 194 ms, total: 2.16 s
Wall time: 1.93 s
3.3576700000
CPU times: user 110 ms, sys: 3.52 ms, total: 114 ms
Wall time: 113 ms


In [24]:
1870/9

207.77777777777777

## Analytical solution

$$\mathbb{E}\left[ D_{n} \right]  = \frac{D_{n-1}}{N} \times  D_{n-1} + \frac{N - D_{n-1}}{N} * (D_{n-1} + 1)$$